# This tutorial is based on a nilearn tutorial with some  modifications.

http://nilearn.github.io/auto_examples/plot_decoding_tutorial.html

In [ ]:
%matplotlib inline

First we need some data.

In [ ]:
from nilearn import datasets
haxby_dataset = datasets.fetch_haxby()
fmri_filename = haxby_dataset.func[0]

# print basic information on the dataset
print('First subject functional nifti images (4D) are at: %s' % fmri_filename)  # 4D data

Let's see what else is in the haxby_dataset variable

In [ ]:
print(haxby_dataset.description)
haxby_dataset

We will use the nilearn.input_data.NiftiMasker to extract the fMRI data on a mask and convert it to data series.

The mask is a mask of the Ventral Temporal streaming coming from the Haxby study:

In [ ]:
mask_filename = haxby_dataset.mask_vt[0]
 
# Let's visualize it, using the subject's anatomical image as a background
from nilearn import plotting
plotting.plot_roi(mask_filename, bg_img=haxby_dataset.anat[0], cmap='Paired')

Now we use the NiftiMasker.

We first create a masker, giving it the options that we care about. Here we use standardizing of the data, as it is often important for decoding

In [ ]:
from nilearn.input_data import NiftiMasker
# Create the Masker with the options
masker = NiftiMasker(mask_img=mask_filename, standardize=True)

# We give the masker a filename and retrieve a 2D array ready
# for machine learning with scikit-learn
fmri_masked = masker.fit_transform(fmri_filename)

The variable "fmri_masked" is a numpy array:

In [ ]:
print(fmri_masked)

Its shape corresponds to the number of TRs times the number of voxels in the mask

In [ ]:
print(fmri_masked.shape)

The behavioral labels are stored in a CSV file, separated by spaces.

We use numpy to load them in an array.



In [ ]:
import numpy as np
# Load target information as string and give a numerical identifier to each
labels = np.recfromcsv(haxby_dataset.session_target[0], delimiter=" ")
print(labels)

Retrieve the behavioral targets, that we are going to predict in the decoding

In [ ]:
target = labels['labels']
print(target)

As we can see from the targets above, the experiment contains many conditions, not all that interest us for decoding.

To keep only data corresponding to faces or cats, we create a mask of the samples belonging to the condition.

In [ ]:
condition_mask = np.logical_or(target == b'face', target == b'cat')

# We apply this mask in the sampe direction to restrict the
# classification to the face vs cat discrimination
fmri_masked = fmri_masked[condition_mask]

We now have less samples



In [ ]:
print(fmri_masked.shape)

We apply the same mask to the targets

In [ ]:
target = target[condition_mask]
print(target.shape)

We will now use the scikit-learn machine-learning toolbox on the fmri_masked data.

As a decoder, we use a Support Vector Classification, with a linear kernel.

We first create it:

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
print(svm)

The svc object is an object that can be fit (or trained) on data with labels, and then predict labels on data without.

We first fit it on the data

In [ ]:
svm.fit(fmri_masked, target)

We can then predict the labels from the data

In [ ]:
prediction = svm.predict(fmri_masked)
print(prediction)

Let?s measure the error rate:

In [ ]:
print((prediction == target).sum() / float(len(target)))

This error rate is meaningless. Why?



The proper way to measure error rates or prediction accuracy is via cross-validation: leaving out some data and testing on it.

Let's leave out the 30 last data points during training, and test the prediction on these 30 last points:

In [ ]:
svm.fit(fmri_masked[:-30], target[:-30])

prediction = svm.predict(fmri_masked[-30:])
print((prediction == target[-30:]).sum() / float(len(target[-30:])))

We can split the data in train and test set repetitively in a KFold strategy:

In [ ]:
from sklearn.cross_validation import KFold

cv = KFold(n=len(fmri_masked), n_folds=5)

for train, test in cv:
    svm.fit(fmri_masked[train], target[train])
    prediction = svm.predict(fmri_masked[test])
    print((prediction == target[test]).sum() / float(len(target[test])))

Scikit-learn has tools to perform cross-validation easier:



In [ ]:
from sklearn.cross_validation import cross_val_score
cv_score = cross_val_score(svm, fmri_masked, target)
print(cv_score)

By default, cross_val_score uses a 3-fold KFold. We can control this by passing the "cv" object, here a 5-fold:

In [ ]:
cv_score = cross_val_score(svm, fmri_masked, target, cv=cv)
print(cv_score)

The best way to do cross-validation is to respect the structure of the experiment, for instance by leaving out full sessions of acquisition.

The number of the session is stored in the CSV file giving the behavioral data. We have to apply our session mask, to select only cats and faces. To leave a session out, we pass it to a LeaveOneLabelOut object:

In [ ]:
session_label = labels['chunks'][condition_mask]

from sklearn.cross_validation import LeaveOneLabelOut
cv = LeaveOneLabelOut(session_label)
cv_score = cross_val_score(svm, fmri_masked, target, cv=cv)
print(cv_score)

Finally, it may be useful to inspect and display the model weights.

We retrieve the SVC discriminating weights



In [ ]:
coef_ = svm.coef_
print(coef_)


In [ ]:
print(coef_.shape)


"coef" is a numpy array.

We need to turn it back into a Nifti image, in essence, "inverting" what the NiftiMasker has done.

For this, we can call inverse_transform on the NiftiMasker:

In [ ]:
coef_img = masker.inverse_transform(coef_)
print(coef_img)



coef_img is now a NiftiImage.

We can save the coefficients as a nii.gz file:



In [ ]:
coef_img.to_filename('haxby_svc_weights.nii.gz')

Also, we can plot the weights, using the subject's anatomical as a background



In [ ]:
from nilearn.plotting import plot_stat_map, show

plot_stat_map(coef_img, bg_img=haxby_dataset.anat[0],
              title="SVM weights", display_mode="yx")

show()